<a href="https://colab.research.google.com/github/ishika-b/neural-networks/blob/main/nnfl_assn2_q10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = '/MyDrive/Colab Notebooks/nnfl2'       # path to your project on Drive

GIT_USERNAME = "ishika-b" # replace with yours
GIT_TOKEN = "43e035e869963c4552e0da695c331cfe8a9248f6"           # definitely replace with yours
GIT_REPOSITORY = "neural-networks"      # ...nah

#drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
#!mkdir "{PROJECT_PATH}"     # in case we haven't created it already   

GIT_PATH = "https://43e035e869963c4552e0da695c331cfe8a9248f6@github.com/ishika-b/neural-networks.git"
!mkdir ./temp
!git clone "https://43e035e869963c4552e0da695c331cfe8a9248f6@github.com/ishika-b/neural-networks.git"
!mv ./temp/* "/content/drive/MyDrive/Colab Notebooks/nnfl2"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

fatal: destination path 'neural-networks' already exists and is not an empty directory.
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
rsync: change_dir "/MyDrive/Colab Notebooks/nnfl2" failed: No such file or directory (2)
rsync error: some files/attrs were not transferred (see previous errors) (code 23) at main.c(1196) [sender=3.1.2]


In [ ]:
!pip install tensorflow 
!pip install keras

In [ ]:
from scipy.io import loadmat
import numpy as np

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

import cv2

In [ ]:
images = []
img = cv2.imread('/content/drive/MyDrive/class1_images.zip (Unzipped Files)/class1_images/class11.png)')
for i in range(237):
  img = cv2.imread('/content/drive/MyDrive/class1_images.zip (Unzipped Files)/class1_images/class1'+ str(i+1)+'.png')
  images.append(img)

In [ ]:
for i in range(252):
  img = cv2.imread('/content/drive/MyDrive/class2_images.zip (Unzipped Files)/class2_images/class2'+ str(i+1)+'.png')
  images.append(img)

for i in range(231):
  img = cv2.imread('/content/drive/MyDrive/class3_images.zip (Unzipped Files)/class3_images/class3'+ str(i+1)+'.png')
  images.append(img)

In [ ]:
np.random.seed(0)
x = np.array(images)
y = np.zeros((720,1))

y[0:237] = 0 
y[237:(237+252)] = 1
y[(237+252):720] = 2

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)

from keras.utils import to_categorical 

y_tr_one_hot  = to_categorical(y_train)
y_te_one_hot  = to_categorical(y_test)
y_val_one_hot = to_categorical(y_val)

In [ ]:
"""
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_val = x_val.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0
x_val = x_val/255.0
"""

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(504, 656, 875, 3)
(144, 656, 875, 3)
(72, 656, 875, 3)


In [ ]:
model = Sequential()
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=3,input_shape=(656, 875, 3))) 
model.add(layers.Conv2D(60, kernel_size=(7,7), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(60, kernel_size=(7,7), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(120, kernel_size=(3,3), activation='relu'))
model.add(layers.Conv2D(120, kernel_size=(3,3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(80, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d (AveragePo (None, 219, 292, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 213, 286, 60)      8880      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 106, 143, 60)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 137, 60)      176460    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 68, 60)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 66, 120)       64920     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 64, 120)       1

In [ ]:
np.random.seed(0)
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
history = model.fit(x_train, y_tr_one_hot, validation_data=(x_val, y_val_one_hot), epochs=50, batch_size=50)

Epoch 1/50
11/11 [==============================] - 3s 274ms/step - loss: 0.6000 - accuracy: 0.8571 - val_loss: 0.5758 - val_accuracy: 0.8750
Epoch 2/50
11/11 [==============================] - 3s 242ms/step - loss: 0.5963 - accuracy: 0.8611 - val_loss: 0.5512 - val_accuracy: 0.8889
Epoch 3/50
11/11 [==============================] - 3s 242ms/step - loss: 0.5776 - accuracy: 0.8552 - val_loss: 0.5311 - val_accuracy: 0.8750
Epoch 4/50
11/11 [==============================] - 3s 242ms/step - loss: 0.5435 - accuracy: 0.8790 - val_loss: 0.5011 - val_accuracy: 0.9028
Epoch 5/50
11/11 [==============================] - 3s 243ms/step - loss: 0.5146 - accuracy: 0.8710 - val_loss: 0.4900 - val_accuracy: 0.8889
Epoch 6/50
11/11 [==============================] - 3s 244ms/step - loss: 0.5162 - accuracy: 0.8671 - val_loss: 0.4662 - val_accuracy: 0.9028
Epoch 7/50
11/11 [==============================] - 3s 244ms/step - loss: 0.4653 - accuracy: 0.9206 - val_loss: 0.4555 - val_accuracy: 0.8889
Epoch 

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_te_one_hot, verbose=0)
print('Accuracy:',test_acc)

Accuracy: 0.9791666865348816


In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = np.argmax(model.predict(np.array(x_test)), axis = -1) + 1
#print(y_pred)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 0 49  0  0]
 [ 0  0 43  3]
 [ 0  0  0 49]
 [ 0  0  0  0]]


In [ ]:
print("class 0 accuracy:",cm[0,1]/(cm[0,1]+cm[0,2]+cm[0,3]) )
print("class 1 accuracy:",cm[1,2]/(cm[1,1]+cm[1,2]+cm[1,3]))
print("class 2 accuracy:",cm[2,3]/(cm[2,1]+cm[2,2]+cm[2,3]) )

class 0 accuracy: 1.0
class 1 accuracy: 0.9347826086956522
class 2 accuracy: 1.0
